In [1]:
from google.colab import drive
drive.mount('/content/drive/')
DIR='/content/drive/MyDrive/Research/OntoConnectWithGNN/GNN_1/Anatomy/'

import os
os.chdir(DIR)

Mounted at /content/drive/


In [2]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

!pip install git+https://github.com/facebookresearch/fastText.git
!pip install git+https://github.com/facebookresearch/PyTorch-BigGraph.git

<IPython.core.display.Javascript object>

1.8.1+cu101
10.1
  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-19oycoin
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-19oycoin
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3090377 sha256=976174216593034700d6e53a8bf3b2a64e35628920ddf9551ef4b790f193c467
  Stored in directory: /tmp/pip-ephem-wheel-cache-cq7dv5zt/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext
  Cloning https://github.com/facebookresearch/PyTorch-BigGraph.git to /tmp/pip-req-build-_urrvrvf
  Running command git clone -q https://github.com/facebookresearch/PyTorch-BigGraph.git /tmp/pip-req-build-_urrvrvf
  Created wheel for torchbiggraph: filename=torchbiggraph-1.0.1.dev0-cp37-none-any.whl size=118771 sha256=6fd4a964efcc87bf8cd6a6d0c569e01affaa08ec316013697169adf14d173ce6
  Stored in directory: /tmp/pip-ephem-wheel-cache-96d3skde/wheels/d5/d0/e8/9fa5

In [3]:
#from OntoSimImports import *
%run OntoSimImports.py

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
#import OntoSimConstants
%run OntoSimConstants.py
from OntoSimConstants import *

In [5]:
def assignVar():
    conf = {
        'gold_copy_fl': DATA_DIR+"/gold_copy/reference.xml",
        'op_fl': DATA_DIR+'/output/output_final.json',
        'txt_pos_fl': DATA_DIR+"/result/pred_op_pos.txt",
        'txt_neg_fl': DATA_DIR+"/result/pred_op_neg.txt"
    }

    return conf

In [6]:
def populategoldCopy(conf):
    gold_copy_fl = conf["gold_copy_fl"]
    gold_copy_root = ET.parse(gold_copy_fl).getroot()
    gold_copy_dict = {}
    for child in gold_copy_root.findall('map/Cell'):
        gold_copy_dict[child[0].get('resource')] = child[1].get('resource') #mouse(MA_)=human(NCI_)'

    return gold_copy_dict

In [7]:
def opCopy(conf):
    with open(code_path+conf['op_fl']) as f:
      op_data = json.load(f)

    op_dict={}
    for op in op_data:
      entity_1=op['entity1']
      entity_2=op['entity2']
      op_dict[entity_1]=entity_2
    
    return op_dict

In [8]:
def testOntoConn(conf, gold_copy_dict, op_dict):
    txt_pos_fl = open(conf["txt_pos_fl"], "a")
    txt_neg_fl = open(conf["txt_neg_fl"], "a")

    A = 0
    R = 0
    tp = 0
    for op in op_dict:
        A = A+1
        if(op in gold_copy_dict):
            gold_op = gold_copy_dict[op]
            if(gold_op in op_dict[op].split(join_str_cnst)):
                tp = tp + 1
                txt_pos_fl.write("Target Key:- "+op+"\n")
                txt_pos_fl.write("Actual OUTPUT:- "+gold_op+"\n")
                txt_pos_fl.write("Predicted OUTPUT:- "+op_dict[op]+"\n")
                txt_pos_fl.write("########################################## \n")
            else:
                txt_neg_fl.write("Target Key:- "+op+"\n")
                txt_neg_fl.write("Actual OUTPUT:- "+gold_op+"\n")
                txt_neg_fl.write("Predicted OUTPUT:- "+op_dict[op]+"\n")
                txt_neg_fl.write("########################################## \n")


    R = len(gold_copy_dict)

    precision = tp / A
    recall = tp / R
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F measure: " + str((2 * precision * recall) / (precision + recall)))

    txt_pos_fl.close()
    txt_neg_fl.close()

In [9]:
def ontoEval(file_arr):
  try:
        print("#################### OntoEvaluation START ####################")
        for fl in file_arr:
          conf = assignVar()
          conf["op_fl"] = DATA_DIR+fl
          print("~~~~~~~~~~~~"+fl)
          gold_copy_dict=populategoldCopy(conf)
          op_dict=opCopy(conf)
          testOntoConn(conf, gold_copy_dict, op_dict)
          print("=====================================")
  except Exception as exp:
    raise exp
  finally:
    print("#################### OntoEvaluation FINISH ####################")

In [10]:
if __name__=="__main__":
  file_arr = ['/output/1_99_output_final.json',
              '/output/1_98_output_final.json',
              '/output/1_97_output_final.json',
              '/output/1_96_output_final.json',
              '/output/1_95_output_final.json',
              '/output/1_94_output_final.json',
              '/output/1_93_output_final.json',
              '/output/1_92_output_final.json',
              '/output/1_91_output_final.json',
              '/output/1_90_output_final.json',
              '/output/3_99_output_final.json',
              '/output/3_98_output_final.json',
              '/output/3_97_output_final.json',
              '/output/3_96_output_final.json',
              '/output/3_95_output_final.json',
              '/output/3_94_output_final.json',
              '/output/3_93_output_final.json',
              '/output/3_92_output_final.json',
              '/output/3_91_output_final.json',
              '/output/3_90_output_final.json',
              '/output/5_99_output_final.json',
              '/output/5_98_output_final.json',
              '/output/5_97_output_final.json',
              '/output/5_96_output_final.json',
              '/output/5_95_output_final.json',
              '/output/5_94_output_final.json',
              '/output/5_93_output_final.json',
              '/output/5_92_output_final.json',
              '/output/5_91_output_final.json',
              '/output/5_90_output_final.json',
           ]
  ontoEval(file_arr)

#################### OntoEvaluation START ####################
~~~~~~~~~~~~/output/1_99_output_final.json
Precision: 0.980411361410382
Recall: 0.6686706746826987
F measure: 0.7950754567116759
~~~~~~~~~~~~/output/1_98_output_final.json
Precision: 0.9761222540592168
Recall: 0.6826987307949232
F measure: 0.8034591194968553
~~~~~~~~~~~~/output/1_97_output_final.json
Precision: 0.9547101449275363
Recall: 0.7040748162992652
F measure: 0.8104575163398693
~~~~~~~~~~~~/output/1_96_output_final.json
Precision: 0.927038626609442
Recall: 0.7214428857715431
F measure: 0.811419984973704
~~~~~~~~~~~~/output/1_95_output_final.json
Precision: 0.8742038216560509
Recall: 0.7334669338677354
F measure: 0.7976752633490737
~~~~~~~~~~~~/output/1_94_output_final.json
Precision: 0.8101083032490974
Recall: 0.749498997995992
F measure: 0.7786259541984732
~~~~~~~~~~~~/output/1_93_output_final.json
Precision: 0.7578176979374585
Recall: 0.7608550434201736
F measure: 0.7593333333333333
~~~~~~~~~~~~/output/1_92_output